In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Utilizando o Gemini para cenários de educação

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/education/use_cases_for_education.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Executar no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/education/use_cases_for_education.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Ver no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/education/use_cases_for_education.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abrir no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão Geral

### Gemini

Gemini é uma família de modelos generativos de IA desenvolvidos pelo Google DeepMind e projetados para casos de uso multimodais. A API Gemini dá acesso aos modelos Gemini Pro Vision e Gemini Pro.

### API Vertex AI Gemini

A API Vertex AI Gemini fornece uma interface unificada para interagir com modelos Gemini. Atualmente existem dois modelos disponíveis na API Gemini:

- **Modelo Gemini Pro** (`gemini-pro`): Projetado para lidar com tarefas de linguagem natural, bate-papo multivoltas de texto e código e geração de código.
- **Modelo Gemini Pro Vision** (`gemini-pro-vision`): Suporta prompts multimodais. Você pode incluir texto, imagens e vídeo em suas solicitações de prompt e obter respostas em texto ou código.

Você pode interagir com a API Gemini usando os seguintes métodos:

- Use o [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para testes rápidos e geração de comandos
- Use o SDK da Vertex AI

Este notebook se concentra no uso do **Vertex AI SDK para Python** para chamar a API Vertex AI Gemini.

Para obter mais informações, consulte a documentação [IA Generativa na Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

O principal objetivo deste notebook é demonstrar uma variedade de casos de uso educacionais que podem se beneficiar dos modelos Gemini.

As etapas executadas incluem:

- Instalando o Python SDK
- Usando a API Vertex AI Gemini
   - Usando um modelo de texto (`gemini-pro`)
     - Raciocínio em diferentes níveis
     - Raciocínio sobre o texto
     - Raciocínio sobre números
   - Usando um modelo multimodal (`gemini-pro-vision`)
     - Raciocínio sobre uma única imagem
     - Raciocínio sobre múltiplas imagens
     - Raciocínio em um vídeo

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.


## Primeiros passos

### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe as bibliotecas necessárias

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.generative_models import (
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part,
)

### Defina algumas funções auxiliares

Defina funções auxiliares para carregar e exibir imagens.

In [ ]:
import http.client
import io
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

Contents = str | list[str | Image | Part]


def generate_content(
    model: GenerativeModel,
    contents: Contents,
    temperature: float = 0.0,
    top_p: float = 0.0,
    top_k: int = 1,
) -> list[GenerationResponse]:
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        candidate_count=1,
        max_output_tokens=2048,
    )

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        stream=True,
    )
    return [responses] if isinstance(responses, GenerationResponse) else list(responses)


def print_contents(contents: Contents):
    if not isinstance(contents, list):
        contents = [contents]

    print(" Conteúdo ".center(80, "-"))
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_image(content)
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_responses(responses: list[GenerationResponse], as_markdown: bool = True):
    # Consolidate the text
    text = "".join(
        part.text
        for response in responses
        for part in response.candidates[0].content.parts
    )
    # Remove potential leading/trailing spaces
    text = text.strip()

    print(" Início das respostas ".center(80, "-"))
    if as_markdown:
        IPython.display.display(IPython.display.Markdown(text))
    else:
        print(text)
    print(" Fim das respostas ".center(80, "-"))
    print("")


def display_image(image: Image, max_width: int = 600, max_height: int = 350):
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # Modes such as RGBA are not yet supported by all Jupyter environments
        pil_image = pil_image.convert("RGB")

    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))

    display_image_compressed(pil_image)


def display_image_compressed(pil_image: PIL_Image.Image):
    """Display the image in a compressed format to reduce the notebook size."""
    image_io = io.BytesIO()
    pil_image.save(image_io, "jpeg", quality=80, optimize=True)
    image_bytes = image_io.getvalue()
    ipython_image = IPython.display.Image(image_bytes)
    IPython.display.display(ipython_image)


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes

## Importe o modelo `Gemini 1.0 Pro`

O Gemini Pro (`gemini-1.0-pro`) ajuda na realização de tarefas utilizando linguagem natural, chats multiturno de texto e código e para a geração de código.

In [ ]:
model = GenerativeModel("gemini-1.0-pro")

### *Reasoning* em diferentes níveis

Você pode realizar perguntas diretas:

In [ ]:
contents = """
O que aconteceu com os dinossauros? Quando?
Explique simplesmente em uma frase.
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

… como também perguntas que requerem respostas com mais nuances:


In [ ]:
contents = """
Temos 100% de certeza sobre o que aconteceu com os dinossauros?
Caso contrário, detalhe as principais hipóteses atuais.
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

Você pode realizar interações pedindo respostas simples:

In [ ]:
contents = """
Explique por que é inverno aqui na França e verão na Austrália.
Explique de uma forma lúdica e engraçada, para que uma criança de 4 anos consiga entender.
Responsa incluindo os 3 motivos principais.
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

… ou pedindo respostas mais detalhadas:


In [ ]:
contents = """
Explique por que temos marés.
Forneça uma resposta detalhada e usando bullets.
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

Você pode realizar perguntas com um escopo bem fechado:

In [ ]:
contents = """
Quando foram os dois últimos anos bissextos?
Liste 3 competições internacionais que aconteceram durante o penúltimo.
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

… e também perguntas mais abertas:

In [ ]:
contents = """
O que veio primeiro, a galinha ou o ovo? Explique a partir de 3 perspectivas diferentes.
O que chamamos de problema do “ovo e da galinha”? Dê 1 exemplo que pode ocorrer na educação.
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

### *Reasoning* com textos

Você pode utilizar o Gemini para realizar sumarizações e traduções:

In [ ]:
contents = """
Resuma o texto a seguir em três frases, em português, usando apenas o texto.

TEXTO:
- Les hommes naissent et demeurent libres et égaux en droits. Les distinctions sociales ne peuvent être fondées que sur l'utilité commune.
- Le but de toute association politique est la conservation des droits naturels et imprescriptibles de l'homme. Ces droits sont la liberté, la propriété, la sûreté et la résistance à l'oppression.
- Le principe de toute souveraineté réside essentiellement dans la Nation. Nul corps, nul individu ne peut exercer d'autorité qui n'en émane expressément.
- La liberté consiste à pouvoir faire tout ce qui ne nuit pas à autrui : ainsi, l'exercice des droits naturels de chaque homme n'a de bornes que celles qui assurent aux autres membres de la société la jouissance de ces mêmes droits. Ces bornes ne peuvent être déterminées que par la loi.
La loi n'a le droit de défendre que les actions nuisibles à la société. Tout ce qui n'est pas défendu par la loi ne peut être empêché, et nul ne peut être contraint à faire ce qu'elle n'ordonne pas.
- La loi est l'expression de la volonté générale. Tous les citoyens ont droit de concourir personnellement ou par leurs représentants à sa formation. Elle doit être la même pour tous, soit qu'elle protège, soit qu'elle punisse. Tous les citoyens, étant égaux à ses yeux, sont également admissibles à toutes dignités, places et emplois publics, selon leur capacité et sans autre distinction que celle de leurs vertus et de leurs talents.
- Nul homme ne peut être accusé, arrêté ou détenu que dans les cas déterminés par la loi et selon les formes qu'elle a prescrites. Ceux qui sollicitent, expédient, exécutent ou font exécuter des ordres arbitraires doivent être punis ; mais tout citoyen appelé ou saisi en vertu de la loi doit obéir à l'instant ; il se rend coupable par la résistance.
- La loi ne doit établir que des peines strictement et évidemment nécessaires, et nul ne peut être puni qu'en vertu d'une loi établie et promulguée antérieurement au délit, et légalement appliquée.
Tout homme étant présumé innocent jusqu'à ce qu'il ait été déclaré coupable, s'il est jugé indispensable de l'arrêter, toute rigueur qui ne serait pas nécessaire pour s'assurer de sa personne doit être sévèrement réprimée par la loi.
- Nul ne doit être inquiété pour ses opinions, même religieuses, pourvu que leur manifestation ne trouble pas l'ordre public établi par la loi.

RESUMO:
-
"""

responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

… como também pedir que o modelo ajude com mais ideias:

In [ ]:
contents = """
Forneça um esboço de 5 pontos-chave para uma apresentação “chocolate no mundo”.
Uma parte deve ser sobre sua origem no México (minha professora tem família lá).
A última será uma degustação com todos da sala de aula.
"""

# For more creative/diverse answers, let's increase the level of randomness.
# Successive requests will likely return different responses.
temperature = 0.7
top_p = 0.8
top_k = 40

responses = generate_content(model, contents, temperature, top_p, top_k)

print_contents(contents)
print_responses(responses)

You can also ask for text corrections:

Below, you can provide some examples of expected responses (i.e. few-shot prompting) so that the model can understand what kind of response you are expecting.


In [ ]:
contents = """
Eu não sou um falante nativo de inglês.
Verifique se as frases a seguir estão corretas.
Quando incorreto, forneça uma correção e uma explicação.
Use a mesma estrutura dos exemplos dados.
EXEMPLOS:
- **Hi!**
  - Status: ✔️
- **Your my best freind!**
  - Status: ❌
  - Correção: **You're my best friend!**
  - Explicação:
    - "**Your**" está incorreto. Parece que você tentou dizer "You're", que é a forma contraída de "You are".
    - "**freind**" é um erro de escrita. A forma correta é "**friend**".

SENTENÇAS:
- They're twins, isn't it?
- I assisted to the meeting.
- You recieved important informations.
- I digged a hole in the ice and saw lots of fishes.
- That's all folks!
"""
responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

… como também pedir contextos mais elaborados e em múltiplos idiomas:

In [ ]:
contents = """
Traduza o texto abaixo para os idiomas listados:

TEXTO:
Hello folks! I hope you're all doing well. Let's get this workshop started!
We'll stick to English because, actually, I can't speak all those languages.

IDIOMAS:
Alemão, Francês, Grego, Búlgaro e Japonês
"""
responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

In [ ]:
contents = """
Não sou falante nativo de inglês e cometi erros nas frases a seguir.
Adivinhe minha língua nativa.
Explique por que esses são erros típicos.
Se houver várias possibilidades, aqui vai uma dica: gosto de queijo.

SENTENÇAS:
- They are twin sisters, isn't it?
- I assisted to the meeting.
- I saw lots of fishes.
"""
responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

### *Reasoning* com números

> Nota: Como qualquer LLM, Gemini gera resultados que parecem plausíveis, mas ainda pode ter alucinações. Dependendo das entradas e dos parâmetros, as saídas podem ser imprecisas, incluindo operações matemáticas. Como prática recomendada, você pode considerar fornecer ao LLM instruções passo a passo para reduzir as alucinações ou usar uma biblioteca de calculadoras em vez de um LLM.

Você pode perguntar sobre problemas da vida real:

In [ ]:
contents = """
Patricia é uma excelente corredora e corre em média 12 km/h.
- Na segunda-feira ela correu por 1,5 hora. Que distância ela correu?
- Na terça ela correu 21 km. Quanto tempo ela correu?
- Na quarta-feira ela correu 150 minutos. Que distância ela correu?
- Em seguida, ela pretende fazer uma maratona (42 km). Quanto deve demorar?
- Para completar uma maratona em 3 horas, quanto mais rápido ela precisa correr?

Detalhe as respostas passo a passo.
"""
responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

… e também sobre problemas mais clássicos:

In [ ]:
contents = """
Acabei de pegar emprestado 1.000 reais de um amigo.
Acordamos uma taxa de juro simples de 4,5% ao ano.
Quero saber quanto terei que reembolsar em 1, 2 ou 3 anos.
"""
responses = generate_content(model, contents)
print_contents(contents)
print_responses(responses)

## Importe o modelo Gemini 1.0 Pro Vision

Gemini Pro Vision (`gemini-1.0-pro-vision`) é um modelo multimodal que suporta prompts multimodais. Você pode incluir texto, imagem(s) e vídeo em suas solicitações de prompt e obter respostas em texto ou código.

In [ ]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### *Reasoning* com uma única imagem


Você pode perguntar coisas simples sobre a imagem:

In [ ]:
prompt = "O que aparece nesta imagem?"
# Image by Crissy Jarvis on Unsplash: https://unsplash.com/photos/cHhbULJbPwM
image_abacus = load_image_from_url(
    "https://unsplash.com/photos/cHhbULJbPwM/download?w=600"
)

contents = [prompt, image_abacus]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

… ou perguntas específicas:

In [ ]:
prompt = """
Responda às seguintes perguntas sobre esta imagem.
Retorne os resultados em formato JSON contendo pares de "perguntas" e "respostas".

QUESTÕES:
- O que a imagem mostra?
- Como funciona?
- Quando foi inventado?
- Qual é o nome deste objeto em francês, italiano, espanhol, holandês e alemão?
- Quais são as cores mais proeminentes na imagem?
"""

contents = [prompt, image_abacus]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

Você também pode incluir perguntas de *follow-up* na requisição:

In [ ]:
# Image by Brett Jordan on Unsplash: https://unsplash.com/photos/E1por_SGvJE
image_tiles = load_image_from_url(
    "https://unsplash.com/photos/E1por_SGvJE/download?w=600"
)
prompt = """
- Que expressão pode ser lida nesta imagem? Como é apresentada?
- Qual é a expressão oposta?
- Qual é a recomendação, a partir desta expressão, que um professor poderia dar aos seus alunos para um exame?
- Com a expressão oposta?
"""

contents = [image_tiles, prompt]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

A informação pode ter diferentes formas. Pode ser objetos, texto impresso, texto manuscrito, etc:

In [ ]:
prompt = """
Siga as instruções.
Escreva expressões matemáticas em LaTex.
Use uma tabela com uma linha para cada instrução e seu resultado.

INSTRUÇÕES:
- Extraia a fórmula.
- Qual é o símbolo logo antes do Pi? O que isso significa?
- Esta é uma fórmula famosa? Isso tem um nome?
- Por que é especial?
- Extraia a legenda.
- Qual é o objeto no fundo?
- Para que foi usado?
- Quais são as cores da legenda e da fórmula?
"""

image_euler = load_image_from_url(
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/math_beauty.jpg"
)

contents = [prompt, image_euler]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

Você também pode solicitar interpretações e sugestões:

In [ ]:
prompt = """
Responda às seguintes perguntas sobre a imagem.
Apresente os resultados em uma tabela com uma linha para cada questão e sua resposta.

QUESTÕES:
- O que é visível?
- Quais são as razões pelas quais é engraçado?
- O que poderia ser uma legenda divertida?
- O que poderia acontecer a seguir?
- Como você alteraria a imagem? Ainda seria engraçado e por quê?
- Como você tornaria isso mais engraçado?
"""

image_classroom = load_image_from_url(
    "https://unsplash.com/photos/4ApmfdVo32Q/download?w=600"
)

contents = [prompt, image_classroom]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

### *Reasoning* com múltiplas imagens

Você também pode utilizar múltiplas imagens:

In [ ]:
prompt = f"""
Responda às seguintes perguntas para cada imagem.
Apresente os resultados em uma tabela com uma linha para cada imagem e uma coluna para cada questão.

QUESTÕES:
- O que podemos ver na imagem?
- Onde se passa?
"""

caption_b1 = "Imagem 1:"
caption_b2 = "Imagem 2:"
caption_b3 = "Imagem 3:"
image_b1 = load_image_from_url("https://unsplash.com/photos/zzjLGF_6dx4/download?w=600")
image_b2 = load_image_from_url("https://unsplash.com/photos/ndAHi2Wxcok/download?w=600")
image_b3 = load_image_from_url("https://unsplash.com/photos/5mZ_M06Fc9g/download?w=600")

contents = [prompt, caption_b1, image_b1, caption_b2, image_b2, caption_b3, image_b3]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

… ou realizar comparações entre imagens:

In [ ]:
prompt = """
Responda às seguintes perguntas sobre as imagens, com uma resposta curta e um motivo detalhado para a resposta.
Apresente os resultados em uma tabela com uma linha para cada pergunta, resposta e motivo.

QUESTÕES:
- O que as imagens têm em comum?
- Qual deles interessaria a um matemático?
- Qual deles indica que é o fim das férias?
- Qual deles sugere que possamos tomar um café lá?
"""

contents = [prompt, caption_b1, image_b1, caption_b2, image_b2, caption_b3, image_b3]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

Você pode usar o nível de linguagem e compreensão visual do Gemini para trabalhar conceitos ou até mesmo obter sugestões de novas imagens:

In [ ]:
prompt = """
Responda às seguintes perguntas sobre as imagens, com uma resposta curta e um motivo detalhado para a resposta.
Apresente os resultados em uma tabela com uma linha para cada pergunta, resposta e motivo.

QUESTÕES:
- O que representa a primeira imagem?
- O que a segunda imagem representa?
- Qual poderia ser a próxima imagem lógica?
"""

caption_w1 = "Imagem 1:"
caption_w2 = "Imagem 2:"
image_w1 = load_image_from_url("https://unsplash.com/photos/TA5bUTySOrg/download?w=600")
image_w2 = load_image_from_url("https://unsplash.com/photos/Nw_D8v79PM4/download?w=600")

contents = [prompt, caption_w1, image_w1, caption_w2, image_w2]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

E o limite é a sua imaginação:

In [ ]:
prompt = """
Responda às seguintes perguntas, com uma resposta curta e um motivo detalhado para a resposta.
Apresente os resultados em uma tabela com uma linha para cada pergunta, resposta e motivo.

QUESTÕES:
- Que tema estas imagens ilustram?
- Qual poderia ser outra imagem para substituir a primeira?
- Que outra imagem poderia substituir a segunda?
- Qual seria uma alternativa à terceira imagem?
- E para o último?
"""

caption_s1 = "Imagem 1:"
caption_s2 = "Imagem 2:"
caption_s3 = "Imagem 3:"
caption_s4 = "Imagem 4:"
image_s1 = load_image_from_url("https://unsplash.com/photos/eriuKJwcdjI/download?w=600")
image_s2 = load_image_from_url("https://unsplash.com/photos/QldMpmrmWuc/download?w=600")
image_s3 = load_image_from_url("https://unsplash.com/photos/rN3m7aTH3io/download?w=600")
image_s4 = load_image_from_url("https://unsplash.com/photos/FhdN5QVrBfY/download?w=600")

contents = [
    prompt,
    caption_s1,
    image_s1,
    caption_s2,
    image_s2,
    caption_s3,
    image_s3,
    caption_s4,
    image_s4,
]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

### *Reasoning* com vídeos

In [ ]:
prompt = """
Responda às seguintes perguntas usando apenas o vídeo.
Apresente os resultados em uma tabela com uma linha para cada questão e sua resposta.

QUESTÕES:
- Qual é o principal animal visível ao longo do vídeo?
- Quais dispositivos eletrônicos estão visíveis?
- Que animais são os personagens de desenhos animados tirando uma selfie em close?
- Quais marcas famosas são visíveis?
- Qual é o texto visível no final?
"""

video = Part.from_uri(
    uri="gs://cloud-samples-data/video/animals.mp4",
    mime_type="video/mp4",
)

contents = [prompt, video]
responses = generate_content(multimodal_model, contents)

print_contents(contents)
print_responses(responses)

## Conclusão

Neste tutorial, você viu exemplos de como usar o Gemini para educação e se beneficiar de modelos textuais e multimodais para gerar conteúdo a partir de textos, imagens e vídeos.

Você também pode explorar outros tutoriais que enfocam diferentes domínios ou especificidades da API Vertex AI Gemini.